In [123]:
from data4ml import Data4ML, Data4Chatbot
from bs4 import BeautifulSoup
import json
import os
import re
from perfect_regex import (
    perfect_url_regex,
    perfect_emoji_regex,
    perfect_email_regex,
    perfect_phone_regex,
)
import random

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
chatbot = Data4Chatbot(path_to_config='./../data_params.json')
chatbot.home_folder = "./data4test"
pairs = chatbot.make_data(limit=0)

100%|██████████| 2/2 [00:00<00:00, 56.47it/s]


In [112]:
pairs

[['привет арин ты уже считала физику ? лабу', 'привет нет еще'],
 ['он разрешил', 'сереж ты спрашивал у екатерины валерьевны про домашку ?'],
 ['сереж ты спрашивал у екатерины валерьевны про домашку ?',
  'ну да \n она мне подсказывала'],
 ['ну да \n она мне подсказывала',
  'ты можешь кинуть формулы если вы их вывели'],
 ['а у тебя какая соль ? \n ну осадок', 'хлорид кальция'],
 ['пришли фотки пожалуйста', 'привет сейчас \n всё'],
 ['какое именно ?', 'последнее . про кино \n спасибо'],
 ['последнее . про кино \n спасибо',
  'скинул \n подойдите завтра к смоленцевой'],
 ['скинул \n подойдите завтра к смоленцевой', 'зачем ?'],
 ['зачем ?', 'она попросила'],
 ['она попросила', 'хорошо'],
 ['пришли фотки пожалуйста', 'привет сейчас \n всё'],
 ['какое именно ?', 'последнее . про кино \n спасибо'],
 ['последнее . про кино \n спасибо',
  'скинул \n подойдите завтра к смоленцевой'],
 ['скинул \n подойдите завтра к смоленцевой', 'зачем ?'],
 ['зачем ?', 'она попросила'],
 ['она попросила', 'хо

In [113]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        """Remove words below a certain count threshold
        """
        
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [114]:
voc = Voc('name')

In [115]:
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

In [116]:
MIN_COUNT = 1    # Minimum word count threshold for trimming


#TODO: Optimize it:
def trimRareWords(voc, pairs, MIN_COUNT):
    """Trim words used under the MIN_COUNT from the voc
    """
    
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 58 / 58 = 1.0000
Trimmed from 17 pairs to 17, 1.0000 of total


In [117]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True) # а нахрена сортировка?
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len




In [163]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[35, 49, 53, 22, 42],
        [17, 50, 24, 23, 43],
        [36, 20, 54, 24, 44],
        [37, 51, 55, 25,  2],
        [38, 24, 56, 26,  0],
        [ 9, 52, 57, 27,  0],
        [24,  2,  2,  2,  0],
        [22,  0,  0,  0,  0],
        [39,  0,  0,  0,  0],
        [ 2,  0,  0,  0,  0]])
lengths: tensor([10,  7,  7,  7,  4])
target_variable: tensor([[40, 53, 58,  5,  3],
        [41, 24,  9, 28, 45],
        [ 2, 54,  2, 29, 24],
        [ 0, 55,  0, 30, 46],
        [ 0, 56,  0, 31,  2],
        [ 0, 57,  0, 32,  0],
        [ 0,  2,  0, 33,  0],
        [ 0,  0,  0, 34,  0],
        [ 0,  0,  0,  2,  0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [False,  True, False,  True,  True],
        [False,  True, False,  True,  True],
        [False,  True, False,  True, False],
        [False,  True, False,  True, False],
        [False, False, False,  True,

In [164]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [165]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [166]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [167]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [169]:
MAX_LENGTH = 10

In [170]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [171]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [172]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [173]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [175]:
device = 'cpu'

In [176]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [178]:
save_dir = 'tmp'

In [180]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, 'lol', loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 4.1050
Iteration: 2; Percent complete: 0.1%; Average loss: 3.9513
Iteration: 3; Percent complete: 0.1%; Average loss: 3.7606
Iteration: 4; Percent complete: 0.1%; Average loss: 3.5510
Iteration: 5; Percent complete: 0.1%; Average loss: 3.3931
Iteration: 6; Percent complete: 0.1%; Average loss: 3.2479
Iteration: 7; Percent complete: 0.2%; Average loss: 3.1744
Iteration: 8; Percent complete: 0.2%; Average loss: 2.9400
Iteration: 9; Percent complete: 0.2%; Average loss: 2.7585
Iteration: 10; Percent complete: 0.2%; Average loss: 2.6449
Iteration: 11; Percent complete: 0.3%; Average loss: 2.4655
Iteration: 12; Percent complete: 0.3%; Average loss: 2.3396
Iteration: 13; Percent complete: 0.3%; Average loss: 2.2048
Iteration: 14; Percent complete: 0.4%; Average loss: 2.0083
Iteration: 15; Percent complete: 0.4%; Average loss: 1.9211
Iteration: 16; Percent complete: 0.4%

KeyboardInterrupt: 

In [96]:
pairs[0]

['последнее . про кино \n спасибо', 'скинул \n подойдите завтра к смоленцевой']

In [100]:
[print(x) for x in pairs]

['последнее . про кино \n спасибо', 'скинул \n подойдите завтра к смоленцевой']
['скинул \n подойдите завтра к смоленцевой', 'зачем ?']
['зачем ?', 'она попросила']
['последнее . про кино \n спасибо', 'скинул \n подойдите завтра к смоленцевой']
['скинул \n подойдите завтра к смоленцевой', 'зачем ?']
['зачем ?', 'она попросила']


[None, None, None, None, None, None]

In [13]:
chatbot = Data4Chatbot(path_to_config='../../../messages')

FileNotFoundError: [Errno 2] No such file or directory: '../../../messages'

In [120]:
list_of_files = chatbot.get_list_of_files_in_folder('data4test/153164714')
print(list_of_files)

['messages0.html']


In [121]:
all_messages = chatbot.parse_html('data4test/153164714', list_of_files)
all_messages

['Юлия Николаева, 23 мар 2016 в 20:48:28\nпришли фотки, пожалуйста',
 'Юлия Николаева, 23 мар 2016 в 20:48:55\nwrong@yandex.ru',
 'Вы, 23 мар 2016 в 21:51:32\nПривет, сейчас',
 'Вы, 23 мар 2016 в 21:53:35\nВсё👌',
 'Юлия Николаева, 23 мар 2016 в 22:50:04\nСпасибо большое',
 'Вы, 23 мар 2016 в 22:53:36\n👌',
 'Юлия Николаева, 20 апр 2016 в 10:39:48\nСережка, перешли мне письмо Смоленцевой.\n1 прикреплённое сообщение',
 'Юлия Николаева, 20 апр 2016 в 10:40:12\nЗаранее спасибо',
 'Вы, 20 апр 2016 в 10:47:08\nКакое именно?',
 'Юлия Николаева, 20 апр 2016 в 10:47:36\nПоследнее. Про кино',
 'Юлия Николаева, 20 апр 2016 в 10:48:44\nСпасибо',
 'Вы, 20 апр 2016 в 10:48:44\nСкинул',
 'Вы, 20 апр 2016 в 22:16:23\nПодойдите завтра к смоленцевой',
 'Юлия Николаева, 20 апр 2016 в 22:20:43\nзачем?',
 'Вы, 21 апр 2016 в 5:26:20\nОна попросила',
 'Юлия Николаева, 21 апр 2016 в 8:37:28\nХорошо',
 'Вы, 22 апр 2016 в 18:36:12\nЮль',
 'Вы, 22 апр 2016 в 18:36:12\nне забывай, что вы завтра в 12 танцуете',
 'В

In [122]:
mes = chatbot.clear_messages(all_messages)
mes

['пришли фотки пожалуйста .',
 'привет сейчас . всё',
 'спасибо большое . сережка перешли мне письмо смоленцевой . заранее спасибо .',
 'какое именно ?',
 'последнее . про кино . спасибо .',
 'скинул . подойдите завтра к смоленцевой .',
 'зачем ?',
 'она попросила .',
 'хорошо .',
 'юль . не забывай что вы завтра в танцуете . и своим передай .',
 'хорошо . спасибо . спасибо . а до скольких ? спасибо .']

In [123]:
chatbot.get_pairs(mes)

[['пришли фотки пожалуйста .', 'привет сейчас . всё'],
 ['какое именно ?', 'последнее . про кино . спасибо .'],
 ['последнее . про кино . спасибо .',
  'скинул . подойдите завтра к смоленцевой .'],
 ['скинул . подойдите завтра к смоленцевой .', 'зачем ?'],
 ['зачем ?', 'она попросила .'],
 ['она попросила .', 'хорошо .']]

In [24]:

def transform_message(message: str) -> str:
    """Make first char upper and add a dot in the end of sentence."""
    # TODO: normilise text

    if message:
        message = message[0].upper() + message[1:]
        if message[-1].isalpha():
            message += '.'
    return message

messages = []
last_author = all_messages[0][:all_messages[0].find(',')] # Человек, начавший беседу

for message in all_messages:
    author =  message[:message.find(',')]
    message = message[message.find("\n") + 1 :]
    clear_message = chatbot._clear_message(message)
    if not clear_message:
        last_author =  author 
        print('lol')
        continue
        
    if author == last_author: # still one message
        if messages:
            messages[-1] += (' ' + transform_message(clear_message))
        else: # for the first iteration
            messages.append(transform_message(clear_message))
    else: # if author change
        messages.append(transform_message(clear_message))

    last_author =  author   

        

lol
lol
lol


In [25]:
messages

['Пришли фотки, пожалуйста.',
 'Привет, сейчас. Всё👌',
 'Спасибо большое.',
 '👌',
 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
 'Какое именно?',
 'Последнее. Про кино. Спасибо.',
 'Скинул. Подойдите завтра к смоленцевой.',
 'Зачем?',
 'Она попросила.',
 'Хорошо.',
 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
 'Хорошо. Спасибо.',
 'Спасибо.',
 'А до скольких? Спасибо.']

In [33]:
messages

[':Юлия Николаева: пришли фотки, пожалуйста',
 ':Вы: Привет, сейчас',
 ':Вы: Всё👌',
 ':Юлия Николаева: Спасибо большое',
 ':Вы: 👌',
 ':Юлия Николаева: Сережка, перешли мне письмо Смоленцевой.',
 ':Юлия Николаева: Заранее спасибо',
 ':Вы: Какое именно?',
 ':Юлия Николаева: Последнее. Про кино',
 ':Юлия Николаева: Спасибо',
 ':Вы: Скинул',
 ':Вы: Подойдите завтра к смоленцевой',
 ':Юлия Николаева: зачем?',
 ':Вы: Она попросила',
 ':Юлия Николаева: Хорошо',
 ':Вы: Юль',
 ':Вы: не забывай, что вы завтра в 12 танцуете',
 ':Вы: И своим передай',
 ':Юлия Николаева: Хорошо',
 ':Юлия Николаева: спасибо',
 ':Юлия Николаева: Спасибо',
 ':Юлия Николаева: а до скольких?',
 ':Юлия Николаева: спасибо']

In [53]:
last_author = re.sub(': .*$', '', messages[0])[1:]
tmp = []

for i in messages:
    author = re.sub(': .*$', '', i)[1:]
    message = re.sub(':.*: ', '', i)
    
    if author == last_author: # значит это всё ещё сообщения от одного пользователя - tmp уже заполнен
        if tmp:
            tmp[-1] += (' ' + transform_message(message))
        else: # для первой итерации
            tmp.append(transform_message(message))
    else: # если пользователь сменился
        tmp.append(transform_message(message))
    
    last_author =  author   
        
tmp

['Пришли фотки, пожалуйста.',
 'Привет, сейчас. Всё👌',
 'Спасибо большое.',
 '👌',
 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
 'Какое именно?',
 'Последнее. Про кино. Спасибо.',
 'Скинул. Подойдите завтра к смоленцевой.',
 'Зачем?',
 'Она попросила.',
 'Хорошо.',
 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
 'Хорошо. Спасибо. Спасибо. А до скольких? Спасибо.']

In [63]:
[[tmp[i-1], tmp[i]] for i in range(1, len(tmp))]

[['Пришли фотки, пожалуйста.', 'Привет, сейчас. Всё👌'],
 ['Привет, сейчас. Всё👌', 'Спасибо большое.'],
 ['Спасибо большое.', '👌'],
 ['👌', 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.'],
 ['Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
  'Какое именно?'],
 ['Какое именно?', 'Последнее. Про кино. Спасибо.'],
 ['Последнее. Про кино. Спасибо.', 'Скинул. Подойдите завтра к смоленцевой.'],
 ['Скинул. Подойдите завтра к смоленцевой.', 'Зачем?'],
 ['Зачем?', 'Она попросила.'],
 ['Она попросила.', 'Хорошо.'],
 ['Хорошо.', 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.'],
 ['Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
  'Хорошо. Спасибо. Спасибо. А до скольких? Спасибо.']]